In [12]:
# ============================================
# Versión simplificada con sliders (VS Code)
# - Sin ipywidgets
# - Sliders y botones de matplotlib
# - Una sola celda, todo comentado
# ============================================

# --- Backend gráfico (elige uno) ---
import matplotlib
matplotlib.use("TkAgg")       # Recomendado en Windows. Abre ventana interactiva.
# matplotlib.use("QtAgg")     # Alternativa (si tienes PyQt5 instalado)

import math
import random
from typing import List, Tuple, Dict, Any

import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.widgets import Slider, Button

# -----------------------------
# 1) Parámetros por defecto
# -----------------------------
DEFAULT_SEED = 42
DEFAULT_C0 = 100.0
DEFAULT_T_DAYS = 365
DEFAULT_LAMBDA = 0.30

# Pools de entidades (de aquí se muestrean 5 al azar)
BANK_POOL   = ["PiBank", "NuBank", "Bankia", "AndesBank", "PacificBank",
               "Aurora", "Atlas", "Vertex", "Nova", "Crescenda"]
STOCK_POOL  = ["AAPL", "TSLA", "AMZN", "GOOGL", "MSFT",
               "NVDA", "META", "NFLX", "JPM", "V", "BABA", "ORCL"]
CRYPTO_POOL = ["BTC", "ETH", "SOL", "ADA", "BNB", "XRP", "DOGE", "DOT"]

# Rangos de parámetros (anualizados)
BANK_RATE_RANGE       = (0.035, 0.065)  # tasa banco
BANK_FIXED_FEE_RANGE  = (0.0, 1.5)      # fee fijo

STOCK_MU_RANGE        = (0.06, 0.12)    # drift acciones
STOCK_SIGMA_RANGE     = (0.15, 0.30)    # vol acciones

CRYPTO_MU_RANGE       = (0.12, 0.30)    # drift cripto
CRYPTO_SIGMA_RANGE    = (0.50, 1.20)    # vol cripto


# ------------------------------------
# 2) Utilidades matemáticas del modelo
# ------------------------------------
def _sample_unique(items: List[str], k: int, rng: random.Random) -> List[str]:
    """Toma k elementos únicos sin reemplazo."""
    return rng.sample(items, k)

def _uniform_in(low: float, high: float, rng: random.Random) -> float:
    """Sortea un float uniforme en [low, high]."""
    return low + rng.random() * (high - low)

def bank_ev_and_std(c0: float, r_annual: float, fee_fixed: float, t_days: int) -> Tuple[float, float]:
    """
    Banco (determinista):
    EV = c0 * (1 + r)^(T/365) - fee, Std = 0
    """
    t_years = t_days / 365.0
    ev = c0 * (1.0 + r_annual) ** t_years - fee_fixed
    return ev, 0.0

def lognormal_ev_and_std(c0: float, mu_annual: float, sigma_annual: float, t_days: int) -> Tuple[float, float]:
    """
    Activo lognormal (acciones/cripto) con retorno continuo:
    X ~ N(m, s^2), m = mu * T_años, s^2 = sigma^2 * T_años
    EV  = c0 * exp(m + 0.5 s^2)
    Var = c0^2 * (exp(s^2) - 1) * exp(2m + s^2)
    """
    t_years = t_days / 365.0
    m = mu_annual * t_years
    s2 = (sigma_annual ** 2) * t_years
    ev = c0 * math.exp(m + 0.5 * s2)
    variance = (c0 ** 2) * (math.exp(s2) - 1.0) * math.exp(2.0 * m + s2)
    std = math.sqrt(variance)
    return ev, std

def score(ev: float, std: float, lam: float) -> float:
    """Puntaje greedy: EV - λ * Std (ajuste por riesgo)."""
    return ev - lam * std


# --------------------------------
# 3) Estado (muestras y parámetros)
# --------------------------------
class GreedyState:
    """Mantiene 5 bancos, 5 acciones, 5 criptos y sus parámetros aleatorios."""
    def __init__(self, seed: int):
        self.seed = int(seed)
        self.rng = random.Random(self.seed)
        self.sample_and_params()

    def sample_and_params(self) -> None:
        # Muestra aleatoria de nombres
        self.banks   = _sample_unique(BANK_POOL,   5, self.rng)
        self.stocks  = _sample_unique(STOCK_POOL,  5, self.rng)
        self.cryptos = _sample_unique(CRYPTO_POOL, 5, self.rng)

        # Parámetros por entidad
        self.bank_params = {
            b: {"rate": _uniform_in(*BANK_RATE_RANGE, self.rng),
                "fee":  _uniform_in(*BANK_FIXED_FEE_RANGE, self.rng)}
            for b in self.banks
        }
        self.stock_params = {
            s: {"mu": _uniform_in(*STOCK_MU_RANGE, self.rng),
                "sigma": _uniform_in(*STOCK_SIGMA_RANGE, self.rng)}
            for s in self.stocks
        }
        self.crypto_params = {
            c: {"mu": _uniform_in(*CRYPTO_MU_RANGE, self.rng),
                "sigma": _uniform_in(*CRYPTO_SIGMA_RANGE, self.rng)}
            for c in self.cryptos
        }


# -------------------------------------
# 4) Evaluación y selección (GREEDY)
# -------------------------------------
def evaluate_and_greedy(state: GreedyState, c0: float, t_days: int, lam: float) -> Dict[str, Any]:
    """Calcula EV/Std/Score por hoja y devuelve la mejor ruta greedy."""
    # Spend (terminal)
    spend_ev, spend_std = c0, 0.0
    spend_score = score(spend_ev, spend_std, lam)

    # Save → mejor banco
    save_rows = []
    for b in state.banks:
        r   = state.bank_params[b]["rate"]
        fee = state.bank_params[b]["fee"]
        ev, std = bank_ev_and_std(c0, r, fee, t_days)
        save_rows.append({"name": b, "ev": ev, "std": std, "score": score(ev, std, lam), "rate": r, "fee": fee})
    best_bank = max(save_rows, key=lambda x: x["score"])

    # Invest → Crypto
    crypto_rows = []
    for c in state.cryptos:
        mu = state.crypto_params[c]["mu"]
        sigma = state.crypto_params[c]["sigma"]
        ev, std = lognormal_ev_and_std(c0, mu, sigma, t_days)
        crypto_rows.append({"name": c, "ev": ev, "std": std, "score": score(ev, std, lam), "mu": mu, "sigma": sigma})
    best_crypto = max(crypto_rows, key=lambda x: x["score"])

    # Invest → Stocks
    stock_rows = []
    for s in state.stocks:
        mu = state.stock_params[s]["mu"]
        sigma = state.stock_params[s]["sigma"]
        ev, std = lognormal_ev_and_std(c0, mu, sigma, t_days)
        stock_rows.append({"name": s, "ev": ev, "std": std, "score": score(ev, std, lam), "mu": mu, "sigma": sigma})
    best_stock = max(stock_rows, key=lambda x: x["score"])

    # Mejor subrama de Invest
    invest_sub = max([best_crypto, best_stock], key=lambda x: x["score"])
    invest_label = "Crypto" if invest_sub["name"] in [r["name"] for r in crypto_rows] else "Stocks"

    # Comparación raíz: Spend vs Save(best) vs Invest(best)
    candidates = [
        ("Spend",  {"ev": spend_ev, "std": spend_std, "score": spend_score}),
        ("Save",   {"ev": best_bank["ev"], "std": best_bank["std"], "score": best_bank["score"], "name": best_bank["name"]}),
        ("Invest", {"ev": invest_sub["ev"], "std": invest_sub["std"], "score": invest_sub["score"], "branch": invest_label, "name": invest_sub["name"]})
    ]
    top_choice = max(candidates, key=lambda kv: kv[1]["score"])

    # Resumen y ruta
    if top_choice[0] == "Spend":
        path = ["Start", "Spend"]
        summary = f"Elegido: Start → Spend | Score={spend_score:.2f} (EV={spend_ev:.2f}, Std={spend_std:.2f})"
    elif top_choice[0] == "Save":
        b = best_bank
        path = ["Start", "Save", b["name"]]
        summary = f"Elegido: Start → Save → {b['name']} | Score={b['score']:.2f} (EV={b['ev']:.2f}, Std={b['std']:.2f})"
    else:
        x = invest_sub
        path = ["Start", "Invest", invest_label, x["name"]]
        summary = f"Elegido: Start → Invest → {invest_label} → {x['name']} | Score={x['score']:.2f} (EV={x['ev']:.2f}, Std={x['std']:.2f})"

    return {
        "summary": summary,
        "path": path,
        "top_choice": top_choice,
        "best_bank": best_bank,
        "best_crypto": best_crypto,
        "best_stock": best_stock,
        "tables": {"save": save_rows, "crypto": crypto_rows, "stock": stock_rows},
    }


# --------------------------------------
# 5) Dibujo del árbol y resaltado GREEDY
# --------------------------------------
def draw_tree(ax, result, spend_ev_for_label=None, c0_for_label=None):
    """
    Dibuja el árbol con layout jerárquico más ancho y etiquetas cortas.
    Compatible con llamadas de 2 ó 4 argumentos.
    """
    ax.clear()

    # --- helpers locales ---
    def short(x, n=1):
        try:
            return f"{float(x):.{n}f}"
        except Exception:
            return str(x)

    def layer_positions(names, y, x0, x1):
        """
        Devuelve {name: (x, y)} con X uniformemente espaciados.
        Robusto si names está vacío o tiene un solo elemento.
        """
        names = list(names)
        if not names:
            return {}
        if len(names) == 1:
            return {names[0]: ((x0 + x1) / 2.0, y)}
        xs = [x0 + i * (x1 - x0) / (len(names) - 1) for i in range(len(names))]
        return {n: (x, y) for n, x in zip(names, xs)}

    # Tablas (si por alguna razón vienen vacías, evitamos crash)
    save_tbl   = list(result.get("tables", {}).get("save", []))
    crypto_tbl = list(result.get("tables", {}).get("crypto", []))
    stock_tbl  = list(result.get("tables", {}).get("stock", []))

    # Info de la elección
    top_choice = result.get("top_choice", ("Spend", {"ev": 0.0, "std": 0.0, "score": 0.0}))
    invest_info = top_choice[1] if isinstance(top_choice, (list, tuple)) and len(top_choice) >= 2 else {}

    # Valor para rotular "Spend"
    c0_label = c0_for_label if c0_for_label is not None else (
               spend_ev_for_label if spend_ev_for_label is not None else
               (top_choice[1].get("ev", 0.0) if isinstance(top_choice, (list, tuple)) and len(top_choice) >= 2 else 0.0)
    )

    # ---- grafo ----
    G = nx.DiGraph()
    G.add_node("root",   label="[D] Start")
    G.add_node("Spend",  label=f"[T] Spend\nEV={short(c0_label,1)}")
    G.add_node("Save",   label="[D] Save")
    G.add_node("Invest", label="[D] Invest")
    G.add_edge("root", "Spend")
    G.add_edge("root", "Save")
    G.add_edge("root", "Invest")

    # Save (bancos)
    for b in save_tbl:
        bn = f"Bank:{b['name']}"
        G.add_node(bn, label=f"[T] {b['name']}\nEV={short(b.get('ev',0),1)}\nrate={short(b.get('rate',0),3)}")
        G.add_edge("Save", bn)

    # Invest → Crypto / Stocks
    G.add_node("Crypto", label="[D] Crypto")
    G.add_node("Stocks", label="[D] Stocks")
    G.add_edge("Invest", "Crypto")
    G.add_edge("Invest", "Stocks")

    for c in crypto_tbl:
        cn = f"Crypto:{c['name']}"
        G.add_node(cn, label=f"[T] {c['name']}\nEV={short(c.get('ev',0),1)}\nμ={short(c.get('mu',0),2)}, σ={short(c.get('sigma',0),2)}")
        G.add_edge("Crypto", cn)

    for s in stock_tbl:
        sn = f"Stock:{s['name']}"
        G.add_node(sn, label=f"[T] {s['name']}\nEV={short(s.get('ev',0),1)}\nμ={short(s.get('mu',0),2)}, σ={short(s.get('sigma',0),2)}")
        G.add_edge("Stocks", sn)

    # ---- layout jerárquico ancho ----
    Y_ROOT, Y_L1, Y_L2, Y_L3 = 4.8, 3.5, 2.0, 0.8
    X_LEFT, X_RIGHT = -7.0, 10.0
    X_SAVE   = (-5.5, -0.5)
    X_CRYPTO = ( 1.0,  4.5)
    X_STOCKS = ( 5.5,  9.5)

    pos = {}
    pos["root"]   = ((X_LEFT + X_RIGHT) / 2.0, Y_ROOT)
    pos["Spend"]  = (X_LEFT + 0.8, Y_L1)
    pos["Save"]   = ((X_SAVE[0] + X_SAVE[1]) / 2.0, Y_L1)
    pos["Invest"] = ((X_CRYPTO[0] + X_STOCKS[1]) / 2.0, Y_L1)

    pos["Crypto"] = ((X_CRYPTO[0] + X_CRYPTO[1]) / 2.0, Y_L2 + 0.5)
    pos["Stocks"] = ((X_STOCKS[0] + X_STOCKS[1]) / 2.0, Y_L2 + 0.5)

    # Hojas por capa
    pos.update(layer_positions([f"Bank:{b['name']}"   for b in save_tbl],   Y_L2, *X_SAVE))
    pos.update(layer_positions([f"Crypto:{c['name']}" for c in crypto_tbl], Y_L3, *X_CRYPTO))
    pos.update(layer_positions([f"Stock:{s['name']}"  for s in stock_tbl],  Y_L3, *X_STOCKS))

    # ---- dibujado ----
    nx.draw(G, pos, with_labels=False, node_size=1800, ax=ax)
    labels = {n: d.get("label", n) for n, d in G.nodes(data=True)}
    nx.draw_networkx_labels(G, pos, labels=labels, font_size=7, ax=ax)

    # ---- resaltar ruta greedy ----
    greedy_edges = []
    if top_choice[0] == "Spend":
        greedy_edges.append(("root", "Spend"))
    elif top_choice[0] == "Save":
        greedy_edges.append(("root", "Save"))
        bb_name = result.get("best_bank", {}).get("name")
        if bb_name:
            greedy_edges.append(("Save", f"Bank:{bb_name}"))
    else:
        greedy_edges.append(("root", "Invest"))
        branch = invest_info.get("branch")
        name   = invest_info.get("name")
        if branch == "Crypto":
            greedy_edges += [("Invest", "Crypto"), ("Crypto", f"Crypto:{name}")]
        elif branch == "Stocks":
            greedy_edges += [("Invest", "Stocks"), ("Stocks", f"Stock:{name}")]

    nx.draw_networkx_edges(G, pos, edgelist=greedy_edges, width=3, ax=ax)

    # Margen y límites
    ax.margins(0.12)
    ax.set_xlim(X_LEFT - 0.5, X_RIGHT + 0.5)
    ax.set_ylim(0.0, Y_ROOT + 0.6)
    ax.set_title("Árbol de decisiones — Ruta GREEDY resaltada")
    ax.axis("off")
# Fin de la sección 5


# ---------------------------------------------------
# 6) App interactiva (sliders + botones) y ejecución
# ---------------------------------------------------
# Creamos la figura con 3 paneles:
#   - ax_tree   : árbol
#   - ax_summary: resumen textual (elección)
#   - ax_log    : run log (historial corto)
fig = plt.figure(figsize=(12, 7))
ax_tree    = plt.axes([0.06, 0.20, 0.62, 0.75])
ax_summary = plt.axes([0.70, 0.65, 0.28, 0.30])
ax_log     = plt.axes([0.70, 0.20, 0.28, 0.40])

# Sliders (parte inferior)
ax_c0   = plt.axes([0.06, 0.12, 0.62, 0.03])
ax_T    = plt.axes([0.06, 0.08, 0.62, 0.03])
ax_lam  = plt.axes([0.06, 0.04, 0.62, 0.03])
ax_seed = plt.axes([0.06, 0.00, 0.62, 0.03])

s_c0   = Slider(ax=ax_c0,   label='Capital C0',      valmin=10.0, valmax=1000.0, valinit=DEFAULT_C0,   valstep=10.0)
s_T    = Slider(ax=ax_T,    label='Horizonte T (d)', valmin=7,    valmax=730,    valinit=DEFAULT_T_DAYS, valstep=1)
s_lam  = Slider(ax=ax_lam,  label='Riesgo λ',        valmin=0.0,  valmax=1.0,    valinit=DEFAULT_LAMBDA, valstep=0.05)
s_seed = Slider(ax=ax_seed, label='Seed',            valmin=1,    valmax=9999,   valinit=DEFAULT_SEED,   valstep=1)

# Botones (derecha)
ax_btn_resample = plt.axes([0.70, 0.58, 0.13, 0.05])
ax_btn_clearlog = plt.axes([0.85, 0.58, 0.13, 0.05])
btn_resample = Button(ax_btn_resample, 'Re-muestrear')
btn_clearlog = Button(ax_btn_clearlog, 'Limpiar log')

# Estado inicial y log
state = GreedyState(int(s_seed.val))
run_log: List[Dict[str, Any]] = []

def append_log(path_str: str, score_val: float) -> None:
    """Agrega una línea al historial (máximo 12 últimas)."""
    run_log.append({
        "seed": int(s_seed.val),
        "C0": float(s_c0.val),
        "T": int(s_T.val),
        "lam": float(s_lam.val),
        "path": path_str,
        "score": float(score_val)
    })
    if len(run_log) > 12:
        del run_log[:-12]

def draw_summary_and_log(result: Dict[str, Any]) -> None:
    """Actualiza paneles de resumen y log."""
    # Resumen
    ax_summary.clear(); ax_summary.axis("off")
    lines = [
        f"Seed: {int(s_seed.val)}",
        f"Bancos:  {state.banks}",
        f"Stocks:  {state.stocks}",
        f"Criptos: {state.cryptos}",
        "",
        result["summary"]
    ]
    ax_summary.text(0.0, 1.0, "\n".join(lines), va="top", ha="left", fontsize=9)

    # Log
    ax_log.clear(); ax_log.axis("off")
    ax_log.text(0.0, 1.0, "Run Log (últimas 12):", va="top", ha="left", fontsize=10)
    if run_log:
        y = 0.93
        for r in reversed(run_log):
            txt = (f"seed={r['seed']} | C0={r['C0']:.0f} | T={r['T']} | λ={r['lam']:.2f} | "
                   f"path={r['path']} | score={r['score']:.2f}")
            ax_log.text(0.0, y, txt, va="top", ha="left", fontsize=8)
            y -= 0.07

def recompute(log_it: bool = True) -> None:
    """Recalcula la decisión greedy y redibuja todo."""
    result = evaluate_and_greedy(state, float(s_c0.val), int(s_T.val), float(s_lam.val))
    draw_tree(ax_tree, result, spend_ev_for_label=float(s_c0.val), c0_for_label=float(s_c0.val))
    path_str  = " → ".join(result["path"])
    score_val = result["top_choice"][1]["score"]
    if log_it:
        append_log(path_str, score_val)
    draw_summary_and_log(result)
    fig.canvas.draw_idle()

# Eventos
def on_resample(event):
    """Botón: re-muestrea entidades y parámetros usando el seed actual."""
    global state
    state = GreedyState(int(s_seed.val))
    recompute(log_it=True)

def on_clearlog(event):
    """Botón: limpia el historial."""
    run_log.clear()
    draw_summary_and_log(evaluate_and_greedy(state, float(s_c0.val), int(s_T.val), float(s_lam.val)))
    fig.canvas.draw_idle()

def on_slider_change(val):
    """Cualquier cambio en sliders: recomputar y loggear."""
    recompute(log_it=True)

# Conectar eventos
btn_resample.on_clicked(on_resample)
btn_clearlog.on_clicked(on_clearlog)

s_c0.on_changed(on_slider_change)
s_T.on_changed(on_slider_change)
s_lam.on_changed(on_slider_change)
s_seed.on_changed(lambda v: on_resample(None))  # cambiar seed -> re-muestrear

# Primer dibujo
recompute(log_it=True)

# Mostrar ventana interactiva (necesario con TkAgg/QtAgg)
plt.show()
# Fin del script :)

In [11]:
# --- VS Code .ipynb launcher (no widget backend needed) ---
import matplotlib
matplotlib.use("TkAgg")     # <-- Try Tk first (usually available on Windows)
import matplotlib.pyplot as plt

# call the function you defined in the previous cell
fig, controls = create_interactive_app()
plt.show()
